# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
from requests.structures import CaseInsensitiveDict
print(geoapify_key)

6e9970a4d6ce4d7b8fc0a8234a08cca2


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port macquarie,-31.4333,152.9167,280.97,91,100,0.89,AU,1721406171
1,1,college,64.8569,-147.8028,286.93,79,20,1.54,US,1721406172
2,2,invercargill,-46.4000,168.3500,278.40,94,77,2.30,NZ,1721406173
3,3,horta,38.5333,-28.6333,298.95,77,20,7.72,PT,1721406035
4,4,udachny,66.4167,112.4000,287.97,61,64,1.81,RU,1721406174


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points('Lng','Lat', geo=True,color='red',alpha=.2, size='Humidity',
xlim=(-180,-30), ylim=(0,55),tiles="OSM")

# Display the map
map_plot1

c:\Users\sayou\anaconda3\anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df['Max Temp']>294) & (city_data_df['Max Temp']<300) & (city_data_df['Wind Speed']< 4.5) & (city_data_df['Cloudiness']==0)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,bethel,41.3712,-73.4140,299.78,62,0,3.09,US,1721406176
103,103,hamilton,39.1834,-84.5333,299.47,52,0,4.12,US,1721406232
126,126,albany,42.6001,-73.9662,298.04,57,0,1.79,US,1721406136
219,219,revelstoke,50.9832,-118.2023,295.16,68,0,1.03,CA,1721406311
228,228,puerto rico,-26.7960,-55.0240,299.31,51,0,2.04,AR,1721406318
309,309,brookings,44.3114,-96.7984,294.47,81,0,3.60,US,1721406321
408,408,sinop,41.7711,34.8709,299.21,46,0,2.70,TR,1721406443
410,410,chokwe,-24.5333,32.9833,295.53,48,0,4.25,MZ,1721406445
477,477,miramichi,47.0043,-65.4654,299.65,35,0,4.28,CA,1721406490
480,480,yarmouth,41.7057,-70.2286,299.60,65,0,3.60,US,1721406492


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City","Country","Lat","Lng","Humidity","Cloudiness"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Cloudiness,Hotel Name
8,bethel,US,41.3712,-73.4140,62,0,
103,hamilton,US,39.1834,-84.5333,52,0,
126,albany,US,42.6001,-73.9662,57,0,
219,revelstoke,CA,50.9832,-118.2023,68,0,
228,puerto rico,AR,-26.7960,-55.0240,51,0,
309,brookings,US,44.3114,-96.7984,81,0,
408,sinop,TR,41.7711,34.8709,46,0,
410,chokwe,MZ,-24.5333,32.9833,48,0,
477,miramichi,CA,47.0043,-65.4654,35,0,
480,yarmouth,US,41.7057,-70.2286,65,0,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
from pprint import pprint
# Set parameters to search for a hotel
params = {
    "radius": 10000,
    "types": "hotel",
    "keyword":"hotels",
    "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]    = ""
    params["bias"]      = f"circle:{longitude},{latitude}"
    

    # Set base URL
    #base_url = "https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20Westminster%20W1H%201LJ%2C%20United%20Kingdom&apiKey=6e9970a4d6ce4d7b8fc0a8234a08cca2"
    base_url = "https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20London%20W1H%201LJ%2C%20United%20Kingdom&format=json&apiKey={geoapify_key}"
    print(base_url)
    
      
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    print(name_address)
    
    results = name_address
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address[0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

 # Display sample data
hotel_df



Starting hotel search
https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20London%20W1H%201LJ%2C%20United%20Kingdom&format=json&apiKey={geoapify_key}
{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}
bethel - nearest hotel: No hotel found
https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20London%20W1H%201LJ%2C%20United%20Kingdom&format=json&apiKey={geoapify_key}
{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}
hamilton - nearest hotel: No hotel found
https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20London%20W1H%201LJ%2C%20United%20Kingdom&format=json&apiKey={geoapify_key}
{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}
albany - nearest hotel: No hotel found
https://api.geoapify.com/v1/geocode/search?text=38%20Upper%20Montagu%20Street%2C%20London%20W1H%201LJ%2C%20United%20Kingdom&format=json&apiKey={geoapify_key}
{'sta

,City,Country,Lat,Lng,Humidity,Cloudiness,Hotel Name
8,bethel,US,41.3712,-73.4140,62,0,No hotel found
103,hamilton,US,39.1834,-84.5333,52,0,No hotel found
126,albany,US,42.6001,-73.9662,57,0,No hotel found
219,revelstoke,CA,50.9832,-118.2023,68,0,No hotel found
228,puerto rico,AR,-26.7960,-55.0240,51,0,No hotel found
309,brookings,US,44.3114,-96.7984,81,0,No hotel found
408,sinop,TR,41.7711,34.8709,46,0,No hotel found
410,chokwe,MZ,-24.5333,32.9833,48,0,No hotel found
477,miramichi,CA,47.0043,-65.4654,35,0,No hotel found
480,yarmouth,US,41.7057,-70.2286,65,0,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Distance",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)